In [1]:
import numpy as np
import pandas as pd
import sklearn

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import average_precision_score, f1_score

from scipy.optimize import minimize_scalar

In [3]:
def get_training_week_data_filename_QD(week_number, simulated_org_number):
    head_folder_name = "/Users/me/Work/FifthQuarterQD/Zhengyang_July4"
    full_filename = "{}/{}/org ({}).csv".format(head_folder_name, 
                                                     week_number, 
                                                     simulated_org_number)
    return full_filename

In [5]:
from pathlib import Path
for week_number in range(5, 52):
    print("can find week {}:{}".format(week_number, 
                                       Path(get_training_week_data_filename_QD(week_number, 9)).exists()))

can find week 5:True
can find week 6:True
can find week 7:True
can find week 8:True
can find week 9:True
can find week 10:True
can find week 11:True
can find week 12:True
can find week 13:True
can find week 14:True
can find week 15:True
can find week 16:True
can find week 17:True
can find week 18:True
can find week 19:True
can find week 20:True
can find week 21:True
can find week 22:True
can find week 23:True
can find week 24:True
can find week 25:True
can find week 26:True
can find week 27:True
can find week 28:True
can find week 29:True
can find week 30:True
can find week 31:True
can find week 32:True
can find week 33:True
can find week 34:True
can find week 35:True
can find week 36:True
can find week 37:True
can find week 38:True
can find week 39:True
can find week 40:True
can find week 41:True
can find week 42:True
can find week 43:True
can find week 44:True
can find week 45:True
can find week 46:True
can find week 47:True
can find week 48:True
can find week 49:True
can find week 5

In [6]:
def read_training_week_data_QD(week_number, simulated_org_number):
    full_filename = get_training_week_data_filename_QD(week_number, simulated_org_number)
    week_df = pd.read_csv(full_filename, index_col = 0) # Note this assumes similar order of users everywhere
    week_df.replace([-np.inf,np.inf], np.nan, inplace=True) #(no matching of users is necessary *under this assumption*)
    return week_df.dropna()

In [7]:
detector_names = read_training_week_data_QD(20, 5).columns.values.tolist()

In [8]:
detector_names

['Target',
 'X001a',
 'X001b',
 'X001c',
 'X014a',
 'X015a',
 'X021a',
 'X021d',
 'X021e',
 'X021f',
 'X021g',
 'X021h',
 'X021i',
 'X021j',
 'X022a',
 'X022d',
 'X022e',
 'X022f',
 'X022g',
 'X022h',
 'X022i',
 'X022j',
 'X027a',
 'X027d',
 'X027e',
 'X027f',
 'X027g',
 'X027h',
 'X027i',
 'X027j',
 'X028a',
 'X028d',
 'X028e',
 'X028f',
 'X028g',
 'X028h',
 'X028i',
 'X028j',
 'X029a',
 'X030a',
 'X031a',
 'X032a',
 'X033a',
 'X034a',
 'X035a',
 'X036a',
 'X037a',
 'X038a',
 'X039a',
 'X040a',
 'X041a',
 'X042a',
 'X043a',
 'X044a',
 'X045a',
 'X046a',
 'X047a',
 'X048a',
 'X049a',
 'X050a',
 'X051a',
 'X052a',
 'X053a',
 'X058a',
 'X059a',
 'X060a']

In [9]:
np.random.seed(662017)

In [10]:
def read_training_org_data(simulated_org_number, first_full_week, last_full_week):
    all_full_week_dfs = []
    list_of_dfs_for_feature_vectors = [read_training_week_data_QD(week_number, simulated_org_number) for week_number in range(first_full_week - 2, first_full_week + 2)]
    for current_week in range(first_full_week, last_full_week + 1):
        list_of_dfs_for_feature_vectors.append(read_week_data_QD(current_week + 2, simulated_org_number))
        current_week_df = pd.concat([list_of_dfs_for_feature_vectors[0].rename(columns = lambda some_str: some_str + "_t-2"), 
                                     list_of_dfs_for_feature_vectors[1].rename(columns = lambda some_str: some_str + "_t-1"), 
                                     list_of_dfs_for_feature_vectors[2].rename(columns = lambda some_str: some_str + "_t"), 
                                     list_of_dfs_for_feature_vectors[3].rename(columns = lambda some_str: some_str + "_t+1"), 
                                     list_of_dfs_for_feature_vectors[4].rename(columns = lambda some_str: some_str + "_t+2")], 
                                    axis = 1)
        current_week_df.dropna(inplace=True)
        all_full_week_dfs.append(current_week_df)
        del list_of_dfs_for_feature_vectors[0]
    return pd.concat(all_full_week_dfs)

From the RCP description:

We first separate the data from the training period (Weeks 7—33) from the data from the testing period (Weeks 34—49). Then, we separate the data from the training period into three distinct groups. The first, $S_1$, is used for training the model and contains 70% of the data. The second, $S_2$, is reserved to use for potential early stopping and contains 10% of the data. The third, $S_3$, is used for validation and contains 20% of the data. Note that we divide the data in a stratified manner so that each group contains both zeros and ones. To be clear, $S_1$ contains 70% of the zeros from the training period and 70% of the ones from the training period. We computed the means and standard deviations of all features over $S_1 \cup S_2$ and used these to standardize all data.

In [11]:
def split_training_data(sample_training_data_df):
    S12_x, S3_x, S12_y, S3_y = train_test_split(sample_training_data_df.drop(['Target_t-2',
                                                                              'Target_t-1',
                                                                              'Target_t',
                                                                              'Target_t+1',
                                                                              'Target_t+2'], 1),
                                                sample_training_data_df['Target_t'],
                                                test_size = 0.2)
    scaler = StandardScaler().fit(S12_x)
    S12_x = scaler.transform(S12_x)
    S3_x = scaler.transform(S3_x)
    S1_x, S2_x, S1_y, S2_y = train_test_split(S12_x, S12_y, test_size = 0.125)
    return S1_x, S2_x, S3_x, S1_y.values, S2_y.values, S3_y.values, scaler

In [14]:
def RCP14_Algorithm_1(S1_x, S2_x, S3_x, S1_y, S2_y, S3_y, iter_per_weight = 25):
    weight_average_areas = {}
    for weight_to_test in range(10, 210, 10):
        weight_areas = np.zeros(iter_per_weight)
        for iteration_num in range(iter_per_weight):
            n_net = MLPRegressor(hidden_layer_sizes=(325,325), 
                                  activation = 'tanh', 
                                  solver = 'sgd', max_iter = 100, early_stopping = True)
            model_prediction_output = get_model_predictions(S1_x, S1_y, S2_x, S2_y, weight_to_test, S3_x)
            weight_areas[iteration_num] = average_precision_score(S3_y, model_prediction_output)
            print("current area is {}".format(weight_areas[iteration_num]))
        weight_average_areas[weight_to_test] = weight_areas.mean()
        print("weight {} had an average area of {}".format(weight_to_test, weight_average_areas[weight_to_test]))
    return max(weight_average_areas.keys(), key=(lambda key: weight_average_areas[key]))

In [15]:
def read_org_test_data(simulated_org_number, first_full_week, last_full_week):
    all_full_week_dfs = []
    list_of_dfs_for_feature_vectors = [read_week_data_QD(week_number, simulated_org_number) for week_number in range(first_full_week - 2, first_full_week + 2)]
    for current_week in range(first_full_week, last_full_week + 1):
        list_of_dfs_for_feature_vectors.append(read_week_data_QD(current_week + 2, simulated_org_number))
        current_week_df = pd.concat([list_of_dfs_for_feature_vectors[0].rename(columns = lambda some_str: some_str + "_t-2"), 
                                     list_of_dfs_for_feature_vectors[1].rename(columns = lambda some_str: some_str + "_t-1"), 
                                     list_of_dfs_for_feature_vectors[2].rename(columns = lambda some_str: some_str + "_t"), 
                                     list_of_dfs_for_feature_vectors[3].rename(columns = lambda some_str: some_str + "_t+1"), 
                                     list_of_dfs_for_feature_vectors[4].rename(columns = lambda some_str: some_str + "_t+2")], 
                                    axis = 1)
        current_week_df.dropna(inplace=True)
        detector_string = 'X021f'
        current_week_df['trait_4'] = ((current_week_df['{}_t-2'.format(detector_string)] > np.percentile(current_week_df['{}_t-2'.format(detector_string)], 90)) | 
                                      (current_week_df['{}_t-1'.format(detector_string)] > np.percentile(current_week_df['{}_t-1'.format(detector_string)], 90)) |
                                      (current_week_df['{}_t'.format(detector_string)] > np.percentile(current_week_df['{}_t'.format(detector_string)], 90)) |
                                      (current_week_df['{}_t+1'.format(detector_string)] > np.percentile(current_week_df['{}_t+1'.format(detector_string)], 90)) |
                                      (current_week_df['{}_t+2'.format(detector_string)] > np.percentile(current_week_df['{}_t+2'.format(detector_string)], 90)))
        detector_string = 'X021h'
        current_week_df['trait_6'] = ((current_week_df['{}_t-2'.format(detector_string)] > np.percentile(current_week_df['{}_t-2'.format(detector_string)], 90)) | 
                                      (current_week_df['{}_t-1'.format(detector_string)] > np.percentile(current_week_df['{}_t-1'.format(detector_string)], 90)) |
                                      (current_week_df['{}_t'.format(detector_string)] > np.percentile(current_week_df['{}_t'.format(detector_string)], 90)) |
                                      (current_week_df['{}_t+1'.format(detector_string)] > np.percentile(current_week_df['{}_t+1'.format(detector_string)], 90)) |
                                      (current_week_df['{}_t+2'.format(detector_string)] > np.percentile(current_week_df['{}_t+2'.format(detector_string)], 90)))
        detector_string = 'X022f'
        current_week_df['trait_8'] = ((current_week_df['{}_t-2'.format(detector_string)] > np.percentile(current_week_df['{}_t-2'.format(detector_string)], 90)) | 
                                      (current_week_df['{}_t-1'.format(detector_string)] > np.percentile(current_week_df['{}_t-1'.format(detector_string)], 90)) |
                                      (current_week_df['{}_t'.format(detector_string)] > np.percentile(current_week_df['{}_t'.format(detector_string)], 90)) |
                                      (current_week_df['{}_t+1'.format(detector_string)] > np.percentile(current_week_df['{}_t+1'.format(detector_string)], 90)) |
                                      (current_week_df['{}_t+2'.format(detector_string)] > np.percentile(current_week_df['{}_t+2'.format(detector_string)], 90)))
        detector_string = 'X022h'
        current_week_df['trait_10'] = ((current_week_df['{}_t-2'.format(detector_string)] > np.percentile(current_week_df['{}_t-2'.format(detector_string)], 90)) | 
                                      (current_week_df['{}_t-1'.format(detector_string)] > np.percentile(current_week_df['{}_t-1'.format(detector_string)], 90)) |
                                      (current_week_df['{}_t'.format(detector_string)] > np.percentile(current_week_df['{}_t'.format(detector_string)], 90)) |
                                      (current_week_df['{}_t+1'.format(detector_string)] > np.percentile(current_week_df['{}_t+1'.format(detector_string)], 90)) |
                                      (current_week_df['{}_t+2'.format(detector_string)] > np.percentile(current_week_df['{}_t+2'.format(detector_string)], 90)))
        detector_string = 'X027f'
        current_week_df['trait_12'] = ((current_week_df['{}_t-2'.format(detector_string)] > np.percentile(current_week_df['{}_t-2'.format(detector_string)], 90)) | 
                                      (current_week_df['{}_t-1'.format(detector_string)] > np.percentile(current_week_df['{}_t-1'.format(detector_string)], 90)) |
                                      (current_week_df['{}_t'.format(detector_string)] > np.percentile(current_week_df['{}_t'.format(detector_string)], 90)) |
                                      (current_week_df['{}_t+1'.format(detector_string)] > np.percentile(current_week_df['{}_t+1'.format(detector_string)], 90)) |
                                      (current_week_df['{}_t+2'.format(detector_string)] > np.percentile(current_week_df['{}_t+2'.format(detector_string)], 90)))
        detector_string = 'X027h'
        current_week_df['trait_14'] = ((current_week_df['{}_t-2'.format(detector_string)] > np.percentile(current_week_df['{}_t-2'.format(detector_string)], 90)) | 
                                      (current_week_df['{}_t-1'.format(detector_string)] > np.percentile(current_week_df['{}_t-1'.format(detector_string)], 90)) |
                                      (current_week_df['{}_t'.format(detector_string)] > np.percentile(current_week_df['{}_t'.format(detector_string)], 90)) |
                                      (current_week_df['{}_t+1'.format(detector_string)] > np.percentile(current_week_df['{}_t+1'.format(detector_string)], 90)) |
                                      (current_week_df['{}_t+2'.format(detector_string)] > np.percentile(current_week_df['{}_t+2'.format(detector_string)], 90)))
        detector_string = 'X028f'
        current_week_df['trait_16'] = ((current_week_df['{}_t-2'.format(detector_string)] > np.percentile(current_week_df['{}_t-2'.format(detector_string)], 90)) | 
                                      (current_week_df['{}_t-1'.format(detector_string)] > np.percentile(current_week_df['{}_t-1'.format(detector_string)], 90)) |
                                      (current_week_df['{}_t'.format(detector_string)] > np.percentile(current_week_df['{}_t'.format(detector_string)], 90)) |
                                      (current_week_df['{}_t+1'.format(detector_string)] > np.percentile(current_week_df['{}_t+1'.format(detector_string)], 90)) |
                                      (current_week_df['{}_t+2'.format(detector_string)] > np.percentile(current_week_df['{}_t+2'.format(detector_string)], 90)))
        detector_string = 'X028h'
        current_week_df['trait_18'] = ((current_week_df['{}_t-2'.format(detector_string)] > np.percentile(current_week_df['{}_t-2'.format(detector_string)], 90)) | 
                                      (current_week_df['{}_t-1'.format(detector_string)] > np.percentile(current_week_df['{}_t-1'.format(detector_string)], 90)) |
                                      (current_week_df['{}_t'.format(detector_string)] > np.percentile(current_week_df['{}_t'.format(detector_string)], 90)) |
                                      (current_week_df['{}_t+1'.format(detector_string)] > np.percentile(current_week_df['{}_t+1'.format(detector_string)], 90)) |
                                      (current_week_df['{}_t+2'.format(detector_string)] > np.percentile(current_week_df['{}_t+2'.format(detector_string)], 90)))
        detector_string = 'X058a'
        current_week_df['trait_20'] = ((current_week_df['{}_t-2'.format(detector_string)]).astype(int) | 
                                      (current_week_df['{}_t-1'.format(detector_string)]).astype(int) |
                                      (current_week_df['{}_t'.format(detector_string)]).astype(int) |
                                      (current_week_df['{}_t+1'.format(detector_string)]).astype(int) |
                                      (current_week_df['{}_t+2'.format(detector_string)]).astype(int))
        all_full_week_dfs.append(current_week_df)
        del list_of_dfs_for_feature_vectors[0]
    return pd.concat(all_full_week_dfs)

In [16]:
all_detector_names = [detector + relative_time for detector in detector_names[1:] for relative_time in ['_t-2', 
                                                                                                        '_t-1', 
                                                                                                        '_t', 
                                                                                                        '_t+1', 
                                                                                                        '_t+2']]
all_trait_names = ["trait_" + str(trait_num) for trait_num in range(4, 21, 2)]

In [17]:
def split_test_data(sample_test_data_df, scaler_from_training_data):
    T_x, T_y, T_generated_attributes = (sample_test_data_df[all_detector_names], 
                                        sample_test_data_df['Target_t'], 
                                        sample_test_data_df[all_trait_names])
    T_x = scaler_from_training_data.transform(T_x)
    return T_x, T_y.values, T_generated_attributes

In [18]:
def calc_f1_score_using_threshold(test_threshold, actual_y, output_for_y):
    return f1_score(actual_y, (output_for_y > test_threshold).astype(int))

In [19]:
def RCP14_Algorithm_2(S1_x, S2_x, S3_x, S1_y, S2_y, S3_y, T_x, T_y, 
                      T_generated_attributes, chosen_weight, num_iterations = 100):
    answer_dict = {"Answer_" + str(answer_num) : np.zeros(num_iterations) for answer_num in range(1, 22)}
    for iteration_num in range(num_iterations):
        current_model = get_model(S1_x, S1_y, S2_x, S2_y, chosen_weight)
        model_prediction_output = current_model.predict(S3_x)
        def func_to_min(x):
            return -calc_f1_score_using_threshold(x, S3_y, model_prediction_output)
        chosen_tau = minimize_scalar(func_to_min, bounds = (0, 1), method = 'bounded').x
        print("optimized cutoff is {}".format(chosen_tau))
        prediction_output_for_test_data = current_model.predict(T_x)
        T_labels = (prediction_output_for_test_data > chosen_tau).astype(int)
        answer_dict["Answer_1"][iteration_num] = (T_y & T_labels).sum() / T_y.sum()
        answer_dict["Answer_2"][iteration_num] = (T_y & T_labels).sum() / T_labels.sum()
        answer_dict["Answer_3"][iteration_num] = (T_y & T_labels).sum() / (T_y ^ 1).sum()
        answer_dict["Answer_4"][iteration_num] = (T_generated_attributes['trait_4'].values & T_labels).mean() / (T_labels).mean()
        answer_dict["Answer_5"][iteration_num] = (T_generated_attributes['trait_4'].values & T_labels).mean() / np.mean(T_generated_attributes['trait_4'])
        answer_dict["Answer_6"][iteration_num] = (T_generated_attributes['trait_6'].values & T_labels).mean() / (T_labels).mean()
        answer_dict["Answer_7"][iteration_num] = (T_generated_attributes['trait_6'].values & T_labels).mean() / np.mean(T_generated_attributes['trait_6'])
        answer_dict["Answer_8"][iteration_num] = (T_generated_attributes['trait_8'].values & T_labels).mean() / (T_labels).mean()
        answer_dict["Answer_9"][iteration_num] = (T_generated_attributes['trait_8'].values & T_labels).mean() / np.mean(T_generated_attributes['trait_8'])
        answer_dict["Answer_10"][iteration_num] = (T_generated_attributes['trait_10'].values & T_labels).mean() / (T_labels).mean()
        answer_dict["Answer_11"][iteration_num] = (T_generated_attributes['trait_10'].values & T_labels).mean() / np.mean(T_generated_attributes['trait_10'])
        answer_dict["Answer_12"][iteration_num] = (T_generated_attributes['trait_12'].values & T_labels).mean() / (T_labels).mean()
        answer_dict["Answer_13"][iteration_num] = (T_generated_attributes['trait_12'].values & T_labels).mean() / np.mean(T_generated_attributes['trait_12'])
        answer_dict["Answer_14"][iteration_num] = (T_generated_attributes['trait_14'].values & T_labels).mean() / (T_labels).mean()
        answer_dict["Answer_15"][iteration_num] = (T_generated_attributes['trait_14'].values & T_labels).mean() / np.mean(T_generated_attributes['trait_14'])
        answer_dict["Answer_16"][iteration_num] = (T_generated_attributes['trait_16'].values & T_labels).mean() / (T_labels).mean()
        answer_dict["Answer_17"][iteration_num] = (T_generated_attributes['trait_16'].values & T_labels).mean() / np.mean(T_generated_attributes['trait_16'])
        answer_dict["Answer_18"][iteration_num] = (T_generated_attributes['trait_18'].values & T_labels).mean() / (T_labels).mean()
        answer_dict["Answer_19"][iteration_num] = (T_generated_attributes['trait_18'].values & T_labels).mean() / np.mean(T_generated_attributes['trait_18'])
        answer_dict["Answer_20"][iteration_num] = (T_generated_attributes['trait_20'].values & T_labels).mean() / (T_labels).mean()
        answer_dict["Answer_21"][iteration_num] = (T_generated_attributes['trait_20'].values & T_labels).mean() / np.mean(T_generated_attributes['trait_20'])
    return answer_dict

In [20]:
def get_all_answers_for_org(simulated_org_number, iter_per_weight = 25, answer_iterations = 100):
    training_data = read_org_data(simulated_org_number, 7, 33)
    training_data.replace([-np.inf,np.inf], np.nan, inplace=True)
    training_data.dropna(inplace=True)
    S1_x, S2_x, S3_x, S1_y, S2_y, S3_y, scaler_from_training_data = split_training_data(training_data)
    weight_to_use = RCP14_Algorithm_1(S1_x, S2_x, S3_x, S1_y, S2_y, S3_y, 
                                      iter_per_weight = iter_per_weight)
    test_data = read_org_test_data(simulated_org_number, 34, 49)
    test_data.replace([-np.inf,np.inf], np.nan, inplace=True)
    test_data.dropna(inplace=True)
    T_x, T_y, T_generated_attributes = split_test_data(test_data, scaler_from_training_data)
    return RCP14_Algorithm_2(S1_x, S2_x, S3_x, S1_y, S2_y, S3_y, T_x, T_y, 
                      T_generated_attributes, weight_to_use, num_iterations = answer_iterations)

In [21]:
# some_df = read_org_data(10, 7, 33)

# some_df.head()

# some_df.shape

# some_df['X028f_t'][~np.isfinite(some_df['X028f_t'])]

# some_test_df = read_org_test_data(10, 34, 49)

# some_test_df['X058a_t'][~np.isfinite(some_test_df['X058a_t+1'])]

In [22]:
Answers_for_each_org_dict = {}

In [ ]:
for org_num in range(1, 13):
    Answers_for_each_org_dict[org_num] = get_all_answers_for_org(org_num)
    pd.DataFrame(Answers_for_each_org_dict[org_num]).to_csv("/home/ec2-user/generated_answers/Answers_Zhengyang_{}.csv".format(org_num))

S1_x has shape (58250, 325)
S2_x has shape (8322, 325)
S3_x has shape (16643, 325)
S1_y has shape (58250,)
S2_y has shape (8322,)
S3_y has shape (16643,)
Train on 58250 samples, validate on 8322 samples
Epoch 1/100
58250/58250 [==============================] - 4s - loss: 0.0725 - val_loss: 0.0087
Epoch 2/100
58250/58250 [==============================] - 4s - loss: 0.0070 - val_loss: 0.0036
Epoch 3/100
58250/58250 [==============================] - 4s - loss: 0.0036 - val_loss: 0.0023
Epoch 4/100
58250/58250 [==============================] - 4s - loss: 0.0023 - val_loss: 0.0016
Epoch 5/100
58250/58250 [==============================] - 4s - loss: 0.0015 - val_loss: 0.0013
Epoch 6/100
58250/58250 [==============================] - 4s - loss: 0.0011 - val_loss: 9.3303e-04
Epoch 7/100
58250/58250 [==============================] - 4s - loss: 8.8765e-04 - val_loss: 7.8763e-04
Epoch 8/100
58250/58250 [==============================] - 4s - loss: 7.2499e-04 - val_loss: 6.4660e-04
Epoch 9/1